In [1]:
import ray
import lsdb
import numpy as np

from ray.util.dask import enable_dask_on_ray, disable_dask_on_ray
client = ray.init(
    num_cpus = 4
)
enable_dask_on_ray()

In [8]:
gaia = lsdb.read_hipscat("/data3/epyc/projects3/ivoa_demo/gaia/catalog")
ztf = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14")

In [10]:
#sources load takes a minute, since it creates a healpix alignment on load
ztf_sources = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source")


In [9]:
result = gaia.cone_search(
    ra=30,
    dec=30,
    radius=1,
)

result.compute()

(dask:cone_filter-c56038f5-6e9d-472b-80e4-d50e8c13fddd pid=126267) /astro/users/atzanida/lsdbgithub/lsdb/src/lsdb/core/cone_search.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(dask:cone_filter-c56038f5-6e9d-472b-80e4-d50e8c13fddd pid=126267)   data_frame["_CONE_SEP"] = df_separations


,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Npix,Dir
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
599782862807367680,1636148068921376768,Gaia DR3 299891766411873280,299891766411873280,1624973553,2016.0,30.325137,0.018750,29.044685,0.019291,2.512269,...,0.1386,0.1118,0.1527,0.0726,0.0586,0.0800,MARCS,3,33,33
599785186384674816,1636148068921376768,Gaia DR3 299892827268818688,299892827268818688,240358398,2016.0,30.447104,0.687952,29.081970,0.717240,1.064708,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
599785336708530176,1636148068921376768,Gaia DR3 299892865923496704,299892865923496704,602695575,2016.0,30.459516,0.165597,29.094229,0.166879,1.533118,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
599785779090161664,1636148068921376768,Gaia DR3 299893140801405312,299893140801405312,614631891,2016.0,30.385253,0.031165,29.071722,0.032536,0.391320,...,0.1545,0.1359,0.1708,0.0833,0.0733,0.0921,MARCS,3,33,33
599785839219703808,1636148068921376768,Gaia DR3 299893175161145344,299893175161145344,557378620,2016.0,30.343850,0.050241,29.055665,0.059042,0.091136,...,0.0864,0.0524,0.1299,0.0467,0.0283,0.0702,MARCS,3,33,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602963402054172672,1636148068921376768,Gaia DR3 301481759014068224,301481759014068224,394261519,2016.0,29.954635,24.048347,30.971129,8.441617,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
602963505133387776,1636148068921376768,Gaia DR3 301481793368995456,301481793368995456,930782920,2016.0,29.972930,0.780689,30.979563,0.719879,0.815885,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
602963552378028032,1636148068921376768,Gaia DR3 301481827728736000,301481827728736000,1423963866,2016.0,29.993989,0.141417,30.983171,0.087812,0.059689,...,0.0028,0.0007,0.0078,0.0015,0.0003,0.0042,MARCS,3,33,33
